In [371]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import StandardScaler

No artigo do Machine Learning Mastery (https://machinelearningmastery.com/model-based-outlier-detection-and-removal-in-python/) são apresentados quatro métodos para tratar outliers. Adicionar os modelos Z-score e DBSCAN utilizando a mesma base de dados e o baseline do artigo. Apresentar os resultados comparando-os com os do artigo.

In [372]:
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv", sep=',', header=None)

In [373]:
#Conhecendo a base de dados
df.shape

(506, 14)

In [374]:
#Conhecendo as variáveis da base de dados
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [375]:
#Visualizando os dados estatísticos
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


As variáveis 0 e 1 apresentam um desvio padrão maior que a média da variável, indicando que estas variáveis contém valores espalhados em uma ampla gama de valores. 

In [376]:
#Separando a base em variáveis de entradas e resposta
df = df.values
X, y = df[:, :-1], df[:, -1]

In [377]:
#Separando a base em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.33, random_state=1)

In [378]:
#Regressão linear sem tratamento de outliers

#Treinando o modelo
model = LinearRegression()
model.fit(X_train, y_train)

#Avaliando o modelo
y_tr = model.predict(X_test)

In [379]:
#Utiliznado a métrica da média de erro absoluto
mae_wout = mean_absolute_error(y_test, y_tr)
print('MAE: ', mae_wout)

MAE:  3.5694358802792885


O resultado da média de erro foi de 3.5694, utilizando a base sem realizar a detecão e remoção dos valores discrepantes.

In [380]:
#Detectando outliers utilizando o Z-score
z = np.abs(stats.zscore(df))

#Selecionando as colunas com valor absoluto menor que 3
filt_result = (z < 3).all(axis=1)

#Criando o dataset sem os outliers
df_z = df[filt_result]
df_z.shape

(415, 14)

O tamanho do dataset, foi reduzido em 91 linhas. Estas linhas apresentavam valores discrepantes em relação ao restante do dataset.

In [381]:
#Divisão da base em Treino e Teste(Z-score)
Xz, yz = df_z[:, :-1], df_z[:, -1]
Xz_train, Xz_test, yz_train, yz_test = train_test_split(Xz, yz, train_size=0.33, random_state=1)

In [382]:
#Executando a regressão linear sem outliers(Z-score)
model = LinearRegression()
model.fit(Xz_train, yz_train)

#Avaliando o modelo
y_tr_z = model.predict(X_test)

In [383]:
#Utiliznado a métrica da média de erro absoluto sem outliers(Z-score)
mae_no_out_z = mean_absolute_error(y_test, y_tr_z)
print('MAE_z: ', mae_no_out_z)

MAE_z:  3.4210537500698965


Houve uma leve melhora na acurácia, ao remover valores discrepantes do dataset. Comparado com a primeira execução que inclui os outliers.

In [384]:
#Divisão da base em Treino e Teste(DBSCAN)
Xd, yd = df[:, :-1], df[:, -1]
Xd_train, Xd_test, yd_train, yd_test = train_test_split(Xd, yd, train_size=0.33, random_state=1)

In [385]:
#Normalizando os dados para treinamento com DBSCAN
ss = StandardScaler()
Xd_train = ss.fit_transform(Xd_train)

#Detectando outliers utilizando o DBSCAN
modelo = DBSCAN(eps=0.8, min_samples=19).fit(Xd_train)

#Quantidade de outliers encontrados
print(Counter(modelo.labels_))

#Visualizando os outliers
filtro = modelo.labels_ == -1
Xd_train_filtered = Xd_train[filtro]

Counter({-1: 166})


Foram encontrados 166 registros no dataset que foram identificados, como fora dos grupos determinados pelos DBSCAN. 

In [386]:
#Executando a regressão linear sem outliers(DBSCAN)
model = LinearRegression()
model.fit(Xd_train_filtered, yd_train)

#Avaliando o modelo
y_tr_d = model.predict(Xd_test)

In [387]:
#Utiliznado a métrica da média de erro absoluto sem outliers(DBSCAN)
mae_no_out_d = mean_absolute_error(yd_test, y_tr_d)
print('MAE_d: ', mae_no_out_d)

MAE_d:  607.3067118430074


O resultado apresentado após aplicação do modelo apresenta um valor inesperado, isso ocorreu devido ao dataset possuir apenas os outliers.

In [388]:
#Comparando a execução entre as três execuções
print('MAE: ', mae_wout)
print('MAE_Z-score: ', mae_no_out_z)
print('MAE_DBSCAN: ', mae_no_out_d)

MAE:  3.5694358802792885
MAE_Z-score:  3.4210537500698965
MAE_DBSCAN:  607.3067118430074


A média de erro absoluto apresentado para a execução da base de dados sem a remoção dos dados apresentou uma leve melhora na acurácia. Este resultado corrobora com a literatura apresentada durantes os estudos, que a remoção de dados tem um baixo impacto no aumento da acurácia dos modelos. O resultado apresentado pelo DBSCAN, apresenta um resultado muito distante do esperado, devido ao conjunto de dados de treinamento utilizado para validar o modelo apresentar apenas dados discrepantes, confirmando que estes dados podem enviesar o resultado dos modelos.
O artigo apresentado utilizou outros algoritmos para identificação automática de outliers, mas o resultado de erro absoluto após remoção destes dados. Apresenta uma leve melhora na acurácia, assim como apresentado neste experimento. 